In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import functools
import os

from matplotlib import gridspec
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from keras.preprocessing import image
import cv2

In [ ]:
def crop_center(image):
  """Returns a cropped square image."""
  shape = image.shape
  new_shape = min(shape[1], shape[2])
  offset_y = max(shape[1] - shape[2], 0) // 2
  offset_x = max(shape[2] - shape[1], 0) // 2
  image = tf.image.crop_to_bounding_box(
      image, offset_y, offset_x, new_shape, new_shape)
  return image

@functools.lru_cache(maxsize=None)
def load_image(image_url, image_size=(256, 256), preserve_aspect_ratio=True):
  """Loads and preprocesses images."""
  # Cache image file locally.
  image_path = image_url
  # Load and convert to float32 numpy array, add batch dimension, and normalize to range [0, 1].
  img = plt.imread(image_path).astype(np.float32)[np.newaxis, ...]
  if img.max() > 1.0:
    img = img / 255.
  if len(img.shape) == 3:
    img = tf.stack([img, img, img], axis=-1)
  img = crop_center(img)
  img = tf.image.resize(img, image_size, preserve_aspect_ratio=True)
  return img

def show_n(images, titles=('',)):
  n = len(images)
  image_sizes = [image.shape[1] for image in images]
  w = (image_sizes[0] * 6) // 320
  plt.figure(figsize=(w  * n, w))
  gs = gridspec.GridSpec(1, n, width_ratios=image_sizes)
  for i in range(n):
    plt.subplot(gs[i])
    plt.imshow(images[i][0], aspect='equal')
    plt.axis('off')
    plt.title(titles[i] if len(titles) > i else '')
  plt.show()

In [ ]:
# Load TF-Hub module.

hub_handle = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'
hub_module = hub.load(hub_handle)

In [ ]:
images = []
names = []
output_image_size = 384  # @param {type:"integer"}
content_img_size = (output_image_size, output_image_size)
style_img_size = (256, 256)
style_image_url = '/content/drive/MyDrive/Colab Notebooks/picture/style/rococo04.jpg'
style_image = load_image(style_image_url, style_img_size)
style_image = tf.nn.avg_pool(style_image, ksize=[3,3], strides=[1,1], padding='SAME')
for i in range(1, 101):
  s = '%06d' % i
  names.append(s)
  content_image_url = '/content/drive/MyDrive/Colab Notebooks/picture/celebrity/' + s + '.jpg'
  # The content image size can be arbitrary.
  content_image = load_image(content_image_url, content_img_size)
  outputs = hub_module(content_image, style_image)
  stylized_image = outputs[0]
  outputs = hub_module(tf.constant(content_image), tf.constant(style_image))
  stylized_image = outputs[0]
  images.append(stylized_image)

show_n(images, names)

In [ ]:
for i in range(100):
  path = '/content/drive/MyDrive/Colab Notebooks/picture/stylized/rococo04/' + names[i] + '.jpg'
  img = tf.image.convert_image_dtype(images[i][0], dtype=tf.uint8)
  cv2.imwrite(path, img.numpy())